## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,28.74,55,19,5.01,few clouds
1,1,Dutse,NG,11.7594,9.3392,63.91,21,0,9.66,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,78.48,74,100,9.06,overcast clouds
3,3,San Quintin,MX,30.4833,-115.9500,65.39,56,0,4.65,clear sky
4,4,Nouadhibou,MR,20.9310,-17.0347,69.78,68,100,11.50,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,78.48,74,100,9.06,overcast clouds
7,7,Avarua,CK,-21.2078,-159.7750,80.65,89,75,9.22,broken clouds
13,13,Yaan,NG,7.3833,8.5667,81.37,17,34,5.88,scattered clouds
14,14,Busselton,AU,-33.6500,115.3333,78.08,57,5,7.96,clear sky
17,17,Sibu,MY,2.3000,111.8167,75.51,100,40,1.99,light rain
22,22,Isangel,VU,-19.5500,169.2667,77.11,94,97,5.75,overcast clouds
23,23,Hervey Bay,AU,-25.2986,152.8535,76.01,78,99,4.00,overcast clouds
26,26,Karratha,AU,-20.7377,116.8463,84.43,66,5,12.59,light rain
27,27,Rosita,NI,13.9268,-84.4032,81.14,72,70,5.44,broken clouds
28,28,Koutiala,ML,12.3917,-5.4642,77.59,12,0,8.63,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                200
City                   200
Country                199
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                199
City                   199
Country                199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.48,overcast clouds,-23.1203,-134.9692,
7,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
13,Yaan,NG,81.37,scattered clouds,7.3833,8.5667,
14,Busselton,AU,78.08,clear sky,-33.6500,115.3333,
17,Sibu,MY,75.51,light rain,2.3000,111.8167,
22,Isangel,VU,77.11,overcast clouds,-19.5500,169.2667,
23,Hervey Bay,AU,76.01,overcast clouds,-25.2986,152.8535,
26,Karratha,AU,84.43,light rain,-20.7377,116.8463,
27,Rosita,NI,81.14,broken clouds,13.9268,-84.4032,
28,Koutiala,ML,77.59,clear sky,12.3917,-5.4642,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.48,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
13,Yaan,NG,81.37,scattered clouds,7.3833,8.5667,
14,Busselton,AU,78.08,clear sky,-33.6500,115.3333,Observatory Guest House
17,Sibu,MY,75.51,light rain,2.3000,111.8167,RH HOTEL (4 Star)


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))